# Loading the reference images

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
references_dir = data_dir / "cloth_competition_references_0002"
teams_dir = data_dir / "evaluation_icra_2024"
grasp_success_csv = data_dir / "evaluation_icra_2024_grasp_success.csv"

# team_name = "AIR_JNU"
# team_name = "Team_Ljubljana"
# team_name = "Ewha_Glab"
team_name = "SCUT-ROBOT"
# team_name = "test_greater_bay"
# team_name = "SRCB-Test"
# team_name = "Shibata_Lab"
# team_name = "ai_robot"
# team_name = "UOS_robotics"
# team_name = "AIS_team"
# team_name = "3c1s"
team_dir = teams_dir / team_name

team_dir

In [ ]:
# import os
# import numpy as np
# from pathlib import Path
# import matplotlib.pyplot as plt

# from cloth_tools.dataset.format import load_competition_observation

# data_dir = Path("data")
# references_dir = data_dir / "cloth_competition_references_0001"
# # references_dir = data_dir / "cloth_competition_references_0002"
# teams_dir = data_dir / "evaluation_icra_2024"
# grasp_success_csv = data_dir / "evaluation_icra_2024_grasp_success.csv"

# # team_name = "AIR_JNU"
# # team_name = "Team_Ljubljana"
# # team_name = "Ewha_Glab"
# team_name = "SCUT-ROBOT"
# # team_name = "test_greater_bay"
# # team_name = "SRCB-Test"
# # team_name = "Shibata_Lab"
# # team_name = "ai_robot"
# # team_name = "UOS_robotics"
# # team_name = "AIS_team"
# # team_name = "3c1s"
# team_dir = teams_dir / team_name

# team_dir

In [ ]:
os.path.exists(references_dir)

In [ ]:
reference_sample_dirs = [references_dir / dirname for dirname in sorted(os.listdir(references_dir))]
reference_sample_dirs

In [ ]:
reference_observation_dirs = [sample_dir / "observation_result" for sample_dir in reference_sample_dirs]
# reference_observation_dirs = reference_sample_dirs # FOR THE OLD REFERENCE FOLDER STRUCTURE

# observation_dirs = [dataset_dir / ref_dir for ref_dir in sorted(os.listdir(dataset_dir))]
len(reference_observation_dirs)

In [ ]:
import json

reference_images = {}
reference_surface_areas = {}

for cloth_id, observation_dir in enumerate(reference_observation_dirs):
    observation = load_competition_observation(observation_dir)

    results_file = observation_dir / "result.json"

    if results_file.exists():
        with open(results_file, "r") as f:
            result = json.load(f)
        surface_area = result["coverage"]
        # reference_images[f"Evaluation reference {cloth_id} ({surface_area:.3f} m²)"] = observation.image_left
        reference_images[f"Evaluation reference {cloth_id}"] = observation.image_left
        reference_surface_areas[cloth_id] = surface_area
    else:
        reference_images[f"Train reference {cloth_id}"] = observation.image_left


In [ ]:
cloth_id = 0
image = reference_images[list(reference_images.keys())[0]]

print(image.shape)



image_cropped = image[20:1020, 660:1660, :] # for references at ICRA
# image_cropped = image[20:1020, 620:1620, :] 

print(image_cropped.shape)

plt.imshow(image_cropped)

In [ ]:
import matplotlib.pyplot as plt

def display_images(images_dict):
    """Displays reference images in a grid layout.

    Args:
        reference_images (dict): A dictionary where keys are cloth IDs and
                                 values are the corresponding images.
    """

    n_images = len(images_dict)
    cols = 4  # Adjust the number of columns as desired (for refs 0002)
    cols = 10  # Adjust the number of columns as desired
    # cols = 2  # Adjust the number of columns as desired
    # cols = 6  # Adjust the number of columns as desired
    rows = (n_images // cols) + (n_images % cols > 0)  # Calculate rows for the grid

    # fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 3, rows * 3.2))  # Create subplots
    # fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 3, rows * 3.5))  # Create subplots
    # fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 3, rows * 3.1))  # Create subplots
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 4, rows * 4))  # Create subplots

    cloth_ids = list(images_dict.keys())
    images = list(images_dict.values())

    placement_dict = {
        0: 0,
        1: 2,
        2: 4,
        3: 6,
        4: 1,
        5: 3,
        6: 5,
        7: 7,
    }

    for i, ax in enumerate(axes.flat):
        if i < n_images:
            # image = images[placement_dict[i]][20:1020, 660:1660, :]
            image = images[i][20:1020, 660:1660, :]
            # image = images[i][20:1020, 620:1620, :]
            # cloth_id = cloth_ids[i]
            ax.imshow(image)
            # ax.set_title(f"{cloth_id}")
            ax.axis("off")  # Hide unused subplots
        # else:

    # plt.tight_layout()
    plt.tight_layout(pad=0.00)

    # plt.subplots_adjust(wspace=0, hspace=0.125)

    plt.show()

# Example usage with your dictionary:
display_images(reference_images) 

In [ ]:
def sample_id_to_reference_id(sample_id):
    return sample_id % 8

def reference_id_to_cloth_type(reference_id):
    if reference_id in [0,1,4,5]:
        return "shirt"
    else:
        return "towel"
    
def reference_id_to_seen(reference_id):
    if reference_id in [0,2,4,6]:
        return True
    else:
        return False

sample_id_to_reference_id(8), sample_id_to_reference_id(10)

In [ ]:
data_dicts = []

team_names = ["AIR_JNU", "Team_Ljubljana", "Ewha_Glab", "SCUT-ROBOT", "test_greater_bay", "SRCB-Test", "Shibata_Lab", "ai_robot", "UOS_robotics", "AIS_team", "3c1s"]

for team_name in team_names:
    team_dir = teams_dir / team_name
    team_sample_dirs = [team_dir / dirname for dirname in sorted(os.listdir(team_dir)) if dirname.startswith("sample_")]

    for i, sample_dir in enumerate(team_sample_dirs):
        
        observation_results_dir = sample_dir / "observation_result"
        
        # Load the surface are
        results_file = observation_results_dir / "result.json"

        with open(results_file, "r") as f:
            result = json.load(f)

        reference_id = sample_id_to_reference_id(i)
        surface_area = result["coverage"]
        reference_surface_area = reference_surface_areas[reference_id]
        coverage = surface_area / reference_surface_area

        data_dict = {
            "team_name": team_name,
            "sample_id": i,
            "reference_id": sample_id_to_reference_id(i),
            "sample_dir": sample_dir,
            "surface_area": surface_area,
            "coverage": coverage,
            "cloth_type": reference_id_to_cloth_type(reference_id),
            "seen": reference_id_to_seen(reference_id),
        }

        data_dicts.append(data_dict)

# convert to pandas dataframe
import pandas as pd

df = pd.DataFrame(data_dicts)

df

In [ ]:
# save df as csv

grasp_success_new_csv = data_dir / "evaluation_icra_2024_grasp_success_new.csv"

df.to_csv(grasp_success_new_csv, index=False)

In [ ]:
# plot a histogram of the coverages
plt.hist(df["coverage"], bins=10)

In [ ]:
plt.hist(df["coverage"], bins=10)



In [ ]:
import matplotlib.pyplot as plt

# Plot the histogram
plt.figure(figsize=(6, 6))  # Adjust size if needed

# Remove the top and right spines (borders)
ax = plt.gca()  # Get the current Axes instance
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.hist(df["coverage"], bins=20, color="skyblue", edgecolor="white")

# Add titles and labels
plt.title("Distribution of Coverages", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Customize appearance
plt.grid(axis='y', alpha=0.75)  # Subtle grid on y-axis
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Show the plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame for each cloth type
shirts = df[df["cloth_type"] == "shirt"]["coverage"]
towels = df[df["cloth_type"] == "towel"]["coverage"]

# Create the figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()  # Get the current Axes for customization

# Plot the histograms
plt.hist(shirts, bins=20, alpha=0.5, label="Shirts", color="skyblue", edgecolor="white")
plt.hist(towels, bins=20, alpha=0.5, label="Towels", color="lightcoral", edgecolor="white")

# Add titles and labels
plt.title("Coverage Distribution by Cloth Type", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)  # Add a legend to distinguish the histograms

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame
seen_items = df[df["seen"]]["coverage"]
unseen_items = df[~df["seen"]]["coverage"]

# Create the figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()

# Plot the histograms
plt.hist(seen_items, bins=20, alpha=0.5, label="Seen", color="green", edgecolor="white")
plt.hist(unseen_items, bins=20, alpha=0.5, label="Unseen", color="purple", edgecolor="white")

# Add titles and labels
plt.title("Coverage Distribution by Seen Status", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
# print average for shirts, towels, seen, unseen
df.groupby("cloth_type")["coverage"].mean()

In [ ]:
df.groupby("seen")["coverage"].mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a function to calculate the percentage above the threshold
def percentage_above_threshold(threshold, coverages):
    return np.mean(coverages >= threshold) * 100  # Calculate percentage

# Create an array of thresholds (0 to 1)
thresholds = np.linspace(0, 1, 100)  # 100 points for smoother curve

# Calculate percentages for each threshold
percentages = [percentage_above_threshold(t, df['coverage']) for t in thresholds]

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(thresholds, percentages, color='skyblue', linewidth=2)

# Add labels and title
plt.title('Percentage of Coverages Above Threshold', fontsize=14)
plt.xlabel('Coverage Threshold', fontsize=12)
plt.ylabel('Percentage Above Threshold (%)', fontsize=12)
plt.grid(axis='y', alpha=0.75)

# Customize appearance (optional)
plt.xlim([0, 1])
plt.ylim([0, 100])
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the function to calculate the percentage above the threshold (same as before)
def percentage_above_threshold(threshold, coverages):
    return np.mean(coverages >= threshold) * 100

# Create an array of thresholds (0 to 1)
thresholds = np.linspace(0, 1, 100)

# Get unique team names
teams = df['team_name'].unique()

# Create the plot
plt.figure(figsize=(10, 6))
for team in teams:
    team_coverages = df[df['team_name'] == team]['coverage']
    percentages = [percentage_above_threshold(t, team_coverages) for t in thresholds]
    plt.plot(thresholds, percentages, label=team)  # Plot one line per team

# Add labels and title
plt.title('Percentage of Coverages Above Threshold by Team', fontsize=14)
plt.xlabel('Coverage Threshold', fontsize=12)
plt.ylabel('Percentage Above Threshold (%)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(axis='y', alpha=0.75)

# Customize appearance (optional)
plt.xlim([0, 1])
plt.ylim([0, 100])
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm  # For colormaps

# Define the function to calculate the percentage above the threshold (same as before)
def percentage_above_threshold(threshold, coverages):
    return np.mean(coverages >= threshold) * 100

# Create an array of thresholds (0 to 1)
thresholds = np.linspace(0, 1, 1000)

# Get unique team names
teams = df['team_name'].unique()

# Choose a colormap with at least 11 distinct colors (tab20 has 20)
cmap = cm.get_cmap('tab20')
colors = [cmap(i) for i in np.linspace(0, 1, len(teams))]  # Evenly spaced colors

# Create the plot
plt.figure(figsize=(12, 7))  # Slightly larger for better legend readability
for i, team in enumerate(teams):
    team_coverages = df[df['team_name'] == team]['coverage']
    percentages = [percentage_above_threshold(t, team_coverages) for t in thresholds]
    plt.plot(thresholds, percentages, label=team, color=colors[i])  # Use the selected color

# Add labels and title
plt.title('Percentage of Coverages Above Threshold by Team', fontsize=14)
plt.xlabel('Coverage Threshold', fontsize=12)
plt.ylabel('Percentage Above Threshold (%)', fontsize=12)
plt.legend(fontsize=10, ncol=2)  # Add legend with 2 columns for better layout

# Customize appearance
plt.xlim([0, 1])
plt.ylim([0, 100])
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', alpha=0.75)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Read the CSV file into a DataFrame
df_grasp_success = pd.read_csv(grasp_success_csv)

df_grasp_success

In [ ]:
df.head()

In [ ]:
import pandas as pd

# Reset Index if Needed: Make sure `df_grasp_success` has a column called 'sample_id'
if not "sample_id" in df_grasp_success.columns:
    df_grasp_success.reset_index(inplace=True, names="sample_id")  

# Melt to Long Format (Key Step): Convert df_grasp_success to a long format for merging
df_grasp_success_long = df_grasp_success.melt(id_vars="sample_id", var_name="team_name", value_name="grasp_success")

# Merge with Left Join: Preserve all rows from 'df' (safer)
merged_df = pd.merge(df, df_grasp_success_long, on=["sample_id", "team_name"], how="left")

team_nice_names = {
    "AIR_JNU": "AIR-jnu",
    "Team_Ljubljana": "Team Ljubljana",
    "Ewha_Glab": "Ewha Glab",
    "SCUT-ROBOT": "SCUT-ROBOT",
    "test_greater_bay": "Team Greater Bay", 
    "SRCB-Test": "SRC - Beijing",
    "Shibata_Lab": "Shibata Lab",
    "ai_robot": "AI&ROBOT LAB",
    "UOS_robotics": "UOS-Robotics",
    "AIS_team": "AIS Shinshu",
    "3c1s": "3C1S"
}


df['team_name'] = df['team_name'].map(team_nice_names)
merged_df['team_name'] = merged_df['team_name'].map(team_nice_names)



merged_df.head(16)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Filter data for success and failure
success_coverages = merged_df[merged_df['grasp_success'] >= 0.5]['coverage']  # Assuming success if >= 0.5
failure_coverages = merged_df[merged_df['grasp_success'] < 0.5]['coverage']

# Create figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()

# Plot histograms with different colors for clarity
plt.hist(success_coverages, bins=20, alpha=0.5, label="Success", color="green", edgecolor="white")
plt.hist(failure_coverages, bins=10, alpha=0.5, label="Failure", color="red", edgecolor="white")

# Add labels, title, and legend
plt.title("Coverage Distribution by Grasp Success", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
import numpy as np

# Filter data for success and failure (same as before)
success_coverages = merged_df[merged_df['grasp_success'] >= 0.5]['coverage']
failure_coverages = merged_df[merged_df['grasp_success'] < 0.5]['coverage']

# Calculate mean and standard deviation
success_mean = np.mean(success_coverages)
success_std = np.std(success_coverages)
failure_mean = np.mean(failure_coverages)
failure_std = np.std(failure_coverages)

# Display the results (formatted for clarity)
print(f"Success - Mean Coverage: {success_mean:.2f}, Std Dev: {success_std:.2f}")
print(f"Failure - Mean Coverage: {failure_mean:.2f}, Std Dev: {failure_std:.2f}")


In [ ]:
# Group by 'team_name' and calculate mean and standard deviation of 'coverage'
team_stats = merged_df.groupby('team_name')['coverage'].agg(['mean', 'std'])

# Rename the columns for clarity
team_stats.columns = ['Mean Coverage', 'Std Dev Coverage']

# Round values to 3 decimal places (optional)
team_stats = team_stats.round(2)

# Display the results
# print(team_stats.to_markdown(numalign="left", stralign="left"))

# Sort by 'Mean Coverage' in ascending order (lowest to highest)
team_stats_sorted = team_stats.sort_values(by='Mean Coverage', ascending=False)

# Display sorted results
print(team_stats_sorted.to_markdown(numalign="left", stralign="left"))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the violin plot
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
sns.violinplot(x="team_name", y="coverage", data=merged_df, density_norm="area")

# Add titles and labels
plt.title("Coverage Distribution by Team (Violin Plot)", fontsize=14)
plt.xlabel("Team", fontsize=12)
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

# Create the violin plot with the cut argument set to 0
plt.figure(figsize=(15, 4))
# Get unique team names
teams = df['team_name'].unique()

# cmap = plt.cm.get_cmap('Set3')
cmap = matplotlib.colormaps["Set3"]
num_teams = len(teams)  # Get the number of teams
colors = list(cmap(range(num_teams)))  # Generate a list of colors from the colormap



# Plot with reversed order using the 'order' parameter
sns.violinplot(x="team_name", y="coverage", data=merged_df, order=teams[::-1], density_norm="area", bw_adjust=0.3) #, palette=colors) #, cut=0)

# for i, team in enumerate(teams[::-1]):  # Reverse the team order
#     sns.violinplot(
#         x="team_name", 
#         y="coverage", 
#         data=merged_df[merged_df['team_name'] == team],  # Filter for each team
#         color=cmap(i),  # Assign color from the colormap
#         bw_adjust=0.3,
#         density_norm="area"
#     )

# Add titles and labels
plt.title("Coverage Distribution by Team (Violin Plot)", fontsize=14)
plt.xlabel("Team", fontsize=12)
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=45) 

# Ensure y-axis limit is set to 1.0
plt.ylim([0, 1.0])

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import joypy
import pandas as pd
# Calculate average coverage for each team
average_coverages = merged_df.groupby('team_name')['coverage'].mean()

# Sort teams by average coverage in descending order
sorted_teams = average_coverages.sort_values(ascending=False).index

# Reverse the sorted teams to have the best teams on top
sorted_teams = sorted_teams[::-1]

# Reindex the DataFrame based on the sorted teams
merged_df_sorted = merged_df.set_index('team_name').loc[sorted_teams].reset_index()

# Group by team with sort=False (use the sorted DataFrame)
grouped = merged_df_sorted.groupby('team_name', sort=False)  

# Create the ridgeline plot, preserving the sorted order
plt.figure(figsize=(12, 8))
joypy.joyplot(
    grouped,
    column=['coverage'],
    overlap=1,
    ylim='own',
    colormap=plt.cm.get_cmap('Set3'),
    alpha=0.7,
    figsize=(10, 6),
    bins=100
)

# Add labels and title
plt.title('Coverage Distribution by Team (Ridgeline Plot - Sorted)', fontsize=14)
plt.xlabel('Coverage', fontsize=12)
plt.show()


In [ ]:
import pandas as pd

# Group by reference_id and calculate average and standard deviation
average_coverage_per_item = merged_df.groupby("reference_id")["coverage"].agg(
    ["mean", "std"]
)

# Rename columns for clarity
average_coverage_per_item.columns = ["Mean Coverage", "Std Dev Coverage"]

# Round values to 2 decimal places
average_coverage_per_item = average_coverage_per_item.round(2)

# Display in a formatted table (markdown)
print(average_coverage_per_item.to_markdown(numalign="left", stralign="left"))



In [ ]:
import pandas as pd

# Define a function to calculate the success rate.
def success_rate(series):
    return (series >= 0.5).mean()

# Group by 'reference_id' and calculate success rate.
success_rate_per_item = merged_df.groupby('reference_id')['grasp_success'].agg(success_rate)

# Rename the column.
success_rate_per_item.name = 'Success Rate'

# Round the values to 3 decimal places and multiply by 100 to get a percentage.
success_rate_per_item = (success_rate_per_item * 100).round(0)

# Print the results in markdown.
print(success_rate_per_item.to_markdown(numalign='left', stralign='left'))


In [ ]:
# Define success condition (grasp_success >= 0.5)
merged_df['success'] = merged_df['grasp_success'] >= 0.5  # assuming 0.5 is the success threshold

# Group by cloth_type and calculate average success rate
average_success_rate_per_cloth_type = merged_df.groupby('cloth_type')['success'].mean()

# Convert to percentages
average_success_rate_per_cloth_type *= 100

# Round results to 2 decimal places
average_success_rate_per_cloth_type = average_success_rate_per_cloth_type.round(0)

# Display the results in markdown
print("Average Grasp Success Rate by Cloth Type:")
print(average_success_rate_per_cloth_type.to_markdown(numalign='left', stralign='left'))


In [ ]:
import pandas as pd

# Create a new column to represent success or failure
merged_df["grasp_result"] = merged_df["grasp_success"].apply(
    lambda x: "Success" if x >= 0.5 else "Failure"
)  # Assuming success if >= 0.5, adjust if needed


# Group by cloth_type and grasp_result, and calculate average coverage
average_coverage_per_cloth_type = merged_df.groupby(["cloth_type", "grasp_result"])[
    "coverage"
].mean()

# Convert Series to DataFrame for better formatting
average_coverage_per_cloth_type_df = average_coverage_per_cloth_type.reset_index()

# Pivot the DataFrame to get grasp_result as columns
average_coverage_per_cloth_type_df = average_coverage_per_cloth_type_df.pivot(
    index="cloth_type", columns="grasp_result", values="coverage"
)

average_coverage_per_cloth_type_df = average_coverage_per_cloth_type_df.round(2)


# Display the results in a table
print(average_coverage_per_cloth_type_df.to_markdown(numalign='left', stralign='left'))


In [ ]:
import pandas as pd

# Define thresholds
thresholds = [0.7, 0.8, 0.9]

# Function to calculate percentage above threshold
def pct_above_threshold(df, threshold):
    return (df['coverage'] > threshold).mean()

# Create an empty DataFrame to store results
percentages_by_team = pd.DataFrame(columns=['team_name'] + thresholds)

# Calculate percentages for each team and threshold
for team in merged_df['team_name'].unique():
    team_data = merged_df.loc[merged_df['team_name'] == team]
    percentages = [pct_above_threshold(team_data, threshold) for threshold in thresholds]
    percentages_by_team.loc[len(percentages_by_team)] = [team] + percentages

# Print the results with labels
print("Percentage of Samples with Coverage Above Thresholds by Team:")
print(percentages_by_team.round(2).to_markdown(index=False, numalign="left", stralign="left"))


In [ ]:
import pandas as pd

# Define thresholds
thresholds = [0.6, 0.7, 0.8, 0.9]

# Function to calculate the count of samples above the threshold
def count_above_threshold(df, threshold):
    return (df['coverage'] > threshold).sum()

# Create an empty DataFrame to store results
counts_by_team = pd.DataFrame(columns=['team_name'] + thresholds)

# Calculate counts for each team and threshold
for team in merged_df['team_name'].unique():
    team_data = merged_df[merged_df['team_name'] == team]
    counts = [count_above_threshold(team_data, threshold) for threshold in thresholds]
    counts_by_team.loc[len(counts_by_team)] = [team] + counts  # Add the team and counts as a new row

# Print the results
print("Count of Samples with Coverage Above Thresholds by Team:")
print(counts_by_team.to_markdown(index=False, numalign="left", stralign="left"))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

team_names = counts_by_team['team_name'].tolist()


# Create a heatmap using seaborn with absolute counts
plt.figure(figsize=(6, 5))
sns.heatmap(
    counts_by_team.drop(columns=["team_name"]),  # Drop the "team" column
    annot=True,
    cmap="Greens",
    fmt="d",  # Format as integers
    linewidths=0.5,
    yticklabels=team_names,
)
plt.title("Count of Samples with Coverage Above Thresholds by Team")
plt.ylabel("Team", fontsize=12)
plt.xlabel("Coverage Threshold", fontsize=12)
# Show the plot
plt.show()


In [ ]:
# from matplotlib.font_manager import get_font_names

# for font in get_font_names():
#     print(font)

team_name_to_method_name = {
    "AIR-jnu": "IGD",
    "Team Ljubljana": "CenDiRNet-6DoF",
    "Ewha Glab": "Sharp Edge Detection",
    "SCUT-ROBOT": "KeypointDetr",
    "Team Greater Bay": "CopGNN",
    "SRC - Beijing": "Grasp-Cloth",
    "Shibata Lab": "Densenet Method",
    "AI&ROBOT LAB": "CFAN",
    "UOS-Robotics": "AED",
    "AIS Shinshu": "Depth2Grasp-CNN",
    "3C1S": "Grasp with PointNet-VAE"
}

df_with_method = merged_df.copy()  # Create a copy to avoid modifying the original
df_with_method['method_name'] = df_with_method['team_name'].map(team_name_to_method_name) 

df_with_method.head(5)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have df_with_method from the previous step

methods = df_with_method['method_name'].unique()[::-1]  # Get unique method names

# Create the swarm plot
plt.figure(figsize=(14, 6), dpi=300)  # Adjust the figure size as needed

sns.violinplot(x="method_name", y="coverage", data=df_with_method, density_norm="area", order=methods, color="#FFFFFF", bw_adjust=0.3, linecolor="#c9c9c9", inner=None)

palette = ["#ed5651", "#4dbf6b"]

sns.swarmplot(x="method_name", y="coverage", data=df_with_method, order=methods, hue='grasp_success', size=8, palette=palette, warn_thresh=0.0)

# Add title and labels
plt.title("Coverage Distribution by Method", fontsize=14)  # Updated title
plt.xlabel("")
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=15) #, ha='right')  # Rotate x-axis labels for better readability

plt.legend(title='Successful grasp', bbox_to_anchor=(1, 1), loc='upper left') 

plt.grid(axis='y', alpha=0.25)  # 'alpha' controls the transparency of the grid lines

# make the y-axis 0 to 1
plt.ylim([0, 1.03])

plt.savefig("coverage_distribution.svg", format="svg") 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have df_with_method from the previous step

methods = df_with_method['method_name'].unique()[::-1]  # Get unique method names

# Create the swarm plot
plt.figure(figsize=(14, 6), dpi=300)  # Adjust the figure size as needed

sns.violinplot(x="method_name", y="coverage", data=df_with_method, density_norm="area", order=methods, color="#FFFFFF", bw_adjust=0.3, linecolor="#c9c9c9", inner=None)

palette = ["#ed5651", "#4dbf6b"]

sns.swarmplot(x="method_name", y="coverage", data=df_with_method, order=methods, hue='seen', size=8, palette=palette, warn_thresh=0.0)

# Add title and labels
plt.title("Coverage Distribution by Method", fontsize=14)  # Updated title
plt.xlabel("")
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=15) #, ha='right')  # Rotate x-axis labels for better readability

plt.legend(title='Seen vs unseen cloth items', bbox_to_anchor=(1, 1), loc='upper left') 

plt.grid(axis='y', alpha=0.25)  # 'alpha' controls the transparency of the grid lines

# make the y-axis 0 to 1
plt.ylim([0, 1.03])

plt.savefig("coverage_distribution.svg", format="svg") 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have df_with_method from the previous step

methods = df_with_method['method_name'].unique()[::-1]  # Get unique method names

df_with_method['succes_and_type'] = df_with_method['grasp_success'].astype(str) + '_' + df_with_method['cloth_type']


# Create the swarm plot
plt.figure(figsize=(14, 6), dpi=300)  # Adjust the figure size as needed

sns.violinplot(x="method_name", y="coverage", data=df_with_method, density_norm="area", order=methods, color="#FFFFFF", bw_adjust=0.3, linecolor="#c9c9c9", inner=None)

# towel colors:
# palette = ["#ed5651", "#4dbf6b"]

palette = ["#f556a5", "#ccff6e", "#ed5651", "#4dbf6b"]

sns.swarmplot(x="method_name", y="coverage", data=df_with_method, order=methods, hue='succes_and_type', size=8, palette=palette, warn_thresh=0.0, marker="s")

# Add title and labels
plt.title("Coverage Distribution by Method", fontsize=14)  # Updated title
plt.xlabel("")
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=15) #, ha='right')  # Rotate x-axis labels for better readability

plt.legend(title='Successful grasp', bbox_to_anchor=(1, 1), loc='upper left') 

plt.grid(axis='y', alpha=0.25)  # 'alpha' controls the transparency of the grid lines

# make the y-axis 0 to 1
plt.ylim([0, 1.03])

plt.savefig("coverage_distribution.svg", format="svg") 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have df_with_method from the previous step

methods = df_with_method['method_name'].unique()[::-1]  # Get unique method names

# Create a new column combining all three factors
df_with_method['combined_factor'] = df_with_method['grasp_result'].astype(str) + '_' + df_with_method['cloth_type'] + '_' + df_with_method['seen'].astype(str)

# Define a palette with 8 distinct colors

# "#4dbf6b"

# f58231 # orange

palette = ["#e6194b", "#f58231", "#ffe119", "#4363d8", "#4dbf6b", "#911eb4", "#46f0f0", "#f032e6"]  # Example colors

# Custom palette: 
# palette = ["#8B0000", "#008000", "#DC143C", "#32CD32", "#006400", "#228B22", "#FF0000", "#B22222"]  # Example colors

# palette = [
#     "#006400",  # Dark Green (Success_Shirt_True)
#     "#008000",  # Green (Success_Shirt_False)
#     "#228B22",  # Forest Green (Success_Towel_True)
#     "#32CD32",  # Lime Green (Success_Towel_False)
#     "#8B0000",  # Dark Red (Failure_Shirt_True)
#     "#B22222",  # Firebrick (Failure_Shirt_False)
#     "#DC143C",  # Crimson (Failure_Towel_True)
#     "#FF0000",  # Red (Failure_Towel_False)
# ]

# Create the swarm plot
plt.figure(figsize=(14, 6), dpi=300)  # Adjust the figure size as needed

sns.violinplot(x="method_name", y="coverage", data=df_with_method, density_norm="area", order=methods, color="#FFFFFF", bw_adjust=0.3, linecolor="#c9c9c9", inner=None)

sns.swarmplot(x="method_name", y="coverage", data=df_with_method, order=methods, hue='combined_factor', size=8, palette=palette, warn_thresh=0.0)

# Add title and labels
plt.title("Coverage Distribution by Method", fontsize=14)  # Updated title
plt.xlabel("")
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=15) #, ha='right')  # Rotate x-axis labels for better readability

# Adjust legend
# handles, labels = plt.gca().get_legend_handles_labels()
# plt.legend(handles, ['Success_shirt_True', 'Success_shirt_False', 'Success_towel_True', 'Success_towel_False', 
#                     'Failure_shirt_True', 'Failure_shirt_False', 'Failure_towel_True', 'Failure_towel_False'], 
#            title='Grasp Status', bbox_to_anchor=(1, 1), loc='upper left') 

plt.legend(title='Grasp Status', bbox_to_anchor=(1, 1), loc='upper left')

plt.grid(axis='y', alpha=0.25)  # 'alpha' controls the transparency of the grid lines

# make the y-axis 0 to 1
plt.ylim([0, 1.03])

plt.savefig("coverage_distribution.svg", format="svg")

In [ ]:
df_with_method.head(5)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

teams = merged_df['team_name'].unique()[::-1] 

# Create the swarm plot
plt.figure(figsize=(14, 6), dpi=300)  # Adjust the figure size as needed

sns.violinplot(x="team_name", y="coverage", data=merged_df, density_norm="area", order=teams, color="#FFFFFF", bw_adjust=0.3, linecolor="#c9c9c9", inner=None)


palette = ["#ed5651", "#4dbf6b"]

sns.swarmplot(x="team_name", y="coverage", data=merged_df, order=teams, hue='grasp_success', size=8, palette=palette, warn_thresh=0.0)

# Add title and labels
plt.title("Coverage Distribution by Team", fontsize=14)
plt.xlabel("")
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=20)  # Rotate x-axis labels if needed

plt.legend(title='Successful grasp', bbox_to_anchor=(1, 1), loc='upper left') 

plt.grid(axis='y', alpha=0.25)  # 'alpha' controls the transparency of the grid lines


# make the y-axis 0 to 1
plt.ylim([0, 1.03])

# ax = plt.gca()


# # Show the plot
# plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Get unique team names and reverse their order
teams = merged_df['team_name'].unique()[::-1] 

# Define markers for each cloth type
markers = {'shirt': 'o', 'towel': 's'}  # You can customize these markers

# Create the swarm plot with the specified order and markers
plt.figure(figsize=(12, 6))

for cloth_type, marker in markers.items():
    sns.swarmplot(
        x="team_name",
        y="coverage",
        data=merged_df[merged_df['cloth_type'] == cloth_type],  # Filter by cloth type
        order=teams,
        marker=marker,
        label=cloth_type,
        size=8
    )

# Add title and labels
plt.title("Coverage Distribution by Team (Swarm Plot)", fontsize=14)
plt.xlabel("Team", fontsize=12)
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=45)

# Show the plot with a legend
plt.legend(title='Cloth Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', alpha=0.75) 
plt.tight_layout()
plt.show()
